In [70]:
import numpy as np
import pandas as pd
from time import sleep
import requests
from tqdm import tqdm
from pprint import pprint
from datetime import datetime
from dotenv import load_dotenv
import os
import time
import random
import string
import csv

In [77]:
# Obtenemos los datos de la aplicación
load_dotenv()
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
REDIRECT_URI = os.getenv("REDIRECT_URI")

In [57]:
#Solicitud del token de acceso
# token de acceso es válido durante 1 hora (3600 segundos).
# Pasado ese tiempo, el token caduca y deberás solicitar uno nuevo.

def solicitar_token():
    # URL de la solicitud POST   
    url = "https://accounts.spotify.com/api/token"

    # Datos del formulario
    payload = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET
    }

    # Encabezados de la solicitud
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }

    # Realizar la solicitud POST
    response = requests.post(url, data=payload, headers=headers)

    access_token=response.json()['access_token']
    tiempo_token=time.time()
    return access_token,tiempo_token



In [72]:
# ENDPOINT SEARCH:
# Solicitud ids
# limit max = 50
# offset max = 1000

def search_in_spotify(access_token,offset,q,type):
    # URL de la solicitud GET
    url = 'https://api.spotify.com/v1/search'

    # Parámetros de la solicitud max 50
    params = {
        'q'     : q ,            # album, artist, track, year,tag:hipster, tag:new, genre, ej: q='year:2023', q= "genres: [chicago rap]"
        'type'  : type,          # type = 'album', 'artist' o 'track'
        'limit' : 50,
        'offset': offset
                        }

    # Encabezados de la solicitud
    headers = {
        "Authorization": f"Bearer {access_token}"
                                                    }

    # Realizar la solicitud GET
    response = requests.get(url, params=params, headers=headers)
    
    # pprint(response.headers) #error 429 ver 'Retry-After'(tiempo de castigo)

    if response.status_code == 200 and response.content:
        return (response.json())
       
    else:
        print(f'Error en get_albums: La respuesta de la API no es un JSON válido. Response={(response,response.headers)} ')




In [73]:
# Búsqueda de álbums. (Se permiten otro tipo de búsquedas, pero se toman los álbums dentro de la estrategia adoptada para extraer la mayor cantidad de información)

def obtener_albums(q):

    data_search = list()
    offset=0
    q    = q           # album, artist, track, year,tag:hipster, tag:new, genre, ej: q='year:2023', q= "genres: [chicago rap]"
    type = 'album'

    for offset in range(0,1000,50):
        data = search_in_spotify(access_token,offset,q,type)
        data_search.extend(data['albums']['items'])
        if data['albums']['next'] == None:
            break 
        sleep(0.15)

    ids_albums=set()
    ids_albums = {(album["id"]) for album in data_search if album}
    return ids_albums

In [56]:
# OBTENCIÓN de ids de álbums. Aquí se genera un set con IDs de albums a partir del SEARCH de SPOTIFY.
#Estos IDs se emplean luego para extraer la info por album, artista y track que se almacena en los DataFrames correspondientes.

# Generamos una cadena de caractéres aleatoria para introducir en 'q' junto al año para obtener resultados de busqueda diferentes 
(access_token,tiempo_token) = solicitar_token()
caracteres = list(set(string.ascii_lowercase).union(set(string.digits)))
year_ini=2020
year_fin=2025
ids_albums_total=set()

# en esta parte se realiza una búsqueda de álbumes con cada uno de los caracteres (por si hay algún nombre de álbum con un solo caracter)
for year in range (year_ini,year_fin):
    for caracter in tqdm(caracteres):
        q = f'year:{year} {caracter}'
        if time.time()-tiempo_token > 3500:
            (access_token,tiempo_token) = solicitar_token()
        ids_albums=obtener_albums(q)
        ids_albums_total = ids_albums_total.union(ids_albums)
        

# en esta parte se realiza una búsqueda de álbumes con un cadena de 2 caractéres aleatoria n veces
for year in range (year_ini,year_fin):
    search_control=[]
    for j in tqdm(range(1000)):
        q = f'year:{year} {random.choice(caracteres)}{random.choice(caracteres)}'
        if time.time()-tiempo_token > 3500:
            (access_token,tiempo_token) = solicitar_token()
        ids_albums=obtener_albums(q)
        
        ids_albums_total = ids_albums_total.union(ids_albums)
       
        # si en 25 búsquedas consecutivas no se aportan nuevos ids al total se para la búsqueda
        search_control.append(len(ids_albums_total))
        if len(search_control)>25 and search_control[-1]==search_control[-25]:
            break

# en esta parte se realiza una búsqueda de álbumes con un cadena de 3 caractéres aleatoria n veces
for year in range (year_ini,year_fin):
    search_control=[]
    for j in tqdm(range(1000)):
        q = f'year:{year} {random.choice(caracteres)}{random.choice(caracteres)}{random.choice(caracteres)}'
        if time.time()-tiempo_token > 3500:
            (access_token,tiempo_token) = solicitar_token()
        ids_albums=obtener_albums(q)
        ids_albums_total = ids_albums_total.union(ids_albums)
        
        # si en 25 búsquedas consecutivas no se aportan nuevos ids al total se para la búsqueda
        search_control.append(len(ids_albums_total))
        if len(search_control)>25 and search_control[-1]==search_control[-25]:
            break
       
            


 19%|█▉        | 7/36 [00:14<00:59,  2.04s/it]


KeyboardInterrupt: 

In [6]:
# Guardar la lista de ids como CSV

# Abrir el archivo en modo de escritura
with open(f'ids_albums_total_{year_ini}_{year_fin}.csv', 'w') as file:
    # Escribir el contenido de la variable en el archivo
    for id in ids_albums_total:
        file.write((id) + '\n')

NameError: name 'year_ini' is not defined

In [12]:
# Abrir el archivo de ids CSV en modo de lectura(por si generamos varias búsquedas de ids)
with open(f'ids_albums_total_2020_2025.csv', 'r', newline='') as file:
    # Crear un objeto lector CSV
    reader = csv.reader(file)
    ids_albums_total=list()
    # Leer cada fila del archivo CSV
    for row in reader:
        # Agregar el ID de álbum a la lista de IDs
        ids_albums_total.append(row[0]) 

In [16]:
data

{'albums': [{'album_type': 'compilation',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'},
     'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of',
     'id': '0LyfQWJT6nXafLPZqxe9Of',
     'name': 'Various Artists',
     'type': 'artist',
     'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
    'LU',
    'MY',
    'MT',
    'MX',
    'NL',
    'NZ',
    'NI',
    'NO',
    'PA',
    'PY',
    'PE',
    'PH',
    'PL',
    'PT',
    'SG',
    'SK',
    'ES',
    'SE',
    'CH',
    'TR',
    'UY',
    'US',
    'GB',
    'AD',
    'LI',
    'MC',
    'ID',
    'JP',
    'TH',
    'VN',
    'RO',
  

In [19]:
len(df_artists)

NameError: name 'df_artists' is not defined

In [14]:
# Generar DataFrames. El proceso es:
    # 1.- Tomamos la los ids_albums_total y lo comparamos con los ids que hay en el DataFrame de albums eliminando los duplicados.
    # 2.- Generar data_albums. Tomamos los ids_albums_total, agrupamos de 20 en 20 (str_ids_albums_20) y hacemos get_albums en bucle
    # 3.- Generar dataframe_albums con el data_albums
    # 4.- Tomamos los ids_artist y los ids_tracks del dataframe_albums
    # 5.- Generar data_artists. Tomamos los ids_artist, los comparamos con los ids que hay en el DataFrame de artist eliminando los duplicados, agrupamos de 100 en 100 (str_ids_artist_100) y hacemos get_artist en bucle
    # 6.- Generar dataframe_artist con el data_artists
    # 7.- Generar data_tracks. Tomamos los ids_albums_total,los comparamos con los ids que hay en el DataFrame de tracks eliminando los duplicados, agrupamos de 100 en 100 (str_ids_tracks_100) y hacemos get_tracks en bucle
    # 8.- Generar dataframe_albums con el data_tracks

# 1.- Tomamos la los ids_albums_total y lo comparamos con los ids que hay en el DataFrame de albums eliminando los duplicados
try:
    df_albums = pd.read_csv('df_albums.csv')
except: 
    data_df = list()
    df_albums = pd.DataFrame(data = data_df, columns = ['genres', 'id', 'images', 'name', 'label','popularity','release_date' ,'release_date_precision' ,'total_tracks' ,'name_tracks','id_tracks' , 'id_artist','name_artist'])
 
ids=set(ids_albums_total)
ids = ids.difference(set(df_albums['id'] ))  #cuidado nombre de variables si lo tomamos desde csv

(access_token,tiempo_token) = solicitar_token()

# 2.- Generar data_albums. Tomamos los ids_albums_total, agrupamos de 20 en 20 (str_ids_albums_20) y hacemos get_albums en bucle
str_ids_albums_20 = str()
offset = 0
ids = list(ids)
data_albums=list()
data=list()
progress_bar = tqdm(total=len(ids) // 21, desc="Processing")

while offset < len(ids):
    str_ids_albums_20 = ','.join( ids[offset:offset+20])
    offset += 20
    
    #comprobar/renovar token
    if time.time()-tiempo_token > 3500:
        (access_token,tiempo_token) = solicitar_token()
    
    #descargamos los datos de los albums de la API y los almacenamos en data_albums
    try:
        data = get_albums(access_token,str_ids_albums_20) 
        data_albums.extend(data.get('albums'))
    except:
        pass    
        progress_bar.update(1)
    sleep(0.15)

progress_bar.close()

# 3.- Completar el dataframe_albums con el data_albums
df_albums_temp = dataframe_albums(data_albums) 
df_albums = pd.concat([df_albums, df_albums_temp], axis = 0)
df_albums.drop_duplicates(subset=['id'], inplace=True)
   
    #guardamos el Dataframe como csv:
df_albums.to_csv('df_albums.csv', index=False)

# 4.- Tomamos los ids_artist y los ids_tracks del dataframe_albums
ids_artist = set(df_albums['id_artist'] )
ids_tracks=set()
for lista in df_albums['id_tracks']:
    ids_tracks.update(lista)

# 5.- Generar data_artists. Tomamos los ids_artist, los comparamos con los ids que hay en el DataFrame de artist eliminando los duplicados, agrupamos de 100 en 100 (str_ids_artist_100) y hacemos get_artist en bucle
try:
    df_artists = pd.read_csv('df_artists.csv')
except: 
    data_df = list()
    df_artists = pd.DataFrame(data = data_df, columns = ['followers', 'genres', 'id', 'images', 'name', 'popularity'])
   
ids=set(ids_artist)
ids = ids.difference(set(df_artists['id'] ))

# Generar data_artists
str_ids_artist_100 = str()
offset = 0
ids = list(ids)
data_artists=list()
data=list()

progress_bar = tqdm(total=(len(ids) // 100)+1, desc="Processing")
while offset < len(ids):
    str_ids_artist_100 = ','.join( ids[offset:offset+100])
    offset += 100
    
    #comprobar/renovar token
    if time.time()-tiempo_token > 3500:
        (access_token,tiempo_token) = solicitar_token()
    
    #descargamos los datos de los albums de la API y los almacenamos en data_albums
    data = get_artists(access_token,str_ids_artist_100) 
    data_artists.extend(data.get('artists'))
    progress_bar.update(1)
    sleep(0.15)
progress_bar.close()

# 6.- Generar dataframe_artist con el data_artists
df_artists_temp = dataframe_artist(data_artists) 
df_artists = pd.concat([df_artists, df_artists_temp], axis = 0)
df_artists.drop_duplicates(subset=['id'], inplace=True)   
 
 #guardamos el Dataframe como csv:
df_artists.to_csv('df_artists.csv', index=False) 

# 7.- Generar data_tracks. Tomamos los ids_albums_total,los comparamos con los ids que hay en el DataFrame de tracks eliminando los duplicados, agrupamos de 100 en 100 (str_ids_tracks_100) y hacemos get_tracks en bucle
try:
    df_tracks = pd.read_csv('df_tracks.csv')
except: 
    data_df = list()
    df_tracks = pd.DataFrame(data = data_df, columns = ['disc_number','duration_ms', 'id', 'name','popularity','track_number','id_album', 'name_album', 'id_artist','name_artist' ])
    
ids=set(ids_tracks)
ids = ids.difference(set(df_tracks['id'] ))

# Generar data_tracks
str_ids_tracks_50 = str()
offset = 0
ids = list(ids)
data_tracks = list()
data = list()
progress_bar = tqdm(total=(len(ids) // 50)+1, desc="Processing")

while offset < len(ids):
    str_ids_tracks_50 = ','.join(ids[offset:offset+50])
    offset += 50
    
    #comprobar/renovar token
    if time.time()-tiempo_token > 3500:
        (access_token,tiempo_token) = solicitar_token()
    
    #descargamos los datos de los albums de la API y los almacenamos en data_tracks
    data = get_tracks(access_token,str_ids_tracks_50)
    data_tracks.extend(data.get('tracks'))
    progress_bar.update(1)
    sleep(0.15)
progress_bar.close()    

# 8.- Generar dataframe_albums con el data_tracks  
df_tracks_temp = dataframe_tracks(data_tracks) 
df_tracks = pd.concat([df_tracks, df_tracks_temp], axis = 0)
df_tracks.drop_duplicates(subset=['id'], inplace=True)
 #guardamos el Dataframe como csv:
df_tracks.to_csv('df_tracks.csv', index=False) 


Processing:   1%|          | 84/11515 [03:02<6:53:24,  2.17s/it]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [31]:
df_albums.head()


,genres,id,images,name,label,popularity,release_date,release_date_precision,total_tracks,name_tracks,id_tracks,id_artist,name_artist
0,NaN,5mVbRaOq4NpjdklSamwaYw,https://i.scdn.co/image/ab67616d0000b2730d718b...,Selskabsmusikerens uundgåelige,John-E-musik,7,2020-06-01,day,15,"['Medley - De gode viser', 'Glad og fri', 'En ...","['79evhBIHNVBeUMxt7QbVjN', '0Y4KrxG78SR8YvpxVn...",7rf1x8dHa91AM74zK5UtSL,John-E-musik
1,NaN,7gqZa9T3Smnmd9dUqzsazf,https://i.scdn.co/image/ab67616d0000b27350721e...,"Relaxing Sounds for Dogs, Vol. 3",Dog Music Jukebox,31,2021-02-28,day,19,"['Sweet Life', 'By My Side', 'Canine Poetry', ...","['16ahe4t9tZiL3eFvGR8bm5', '0kkzZ7wlxCSHZ8oKdg...",4s8jebxAHyT9D8oTCxJdVw,Dog Music Jukebox
2,NaN,4Wl8ucOExTyer5104pKGQD,https://i.scdn.co/image/ab67616d0000b273e7ed52...,La Machina,RwR Records,10,2020-09-25,day,1,['La Machina'],['47srvTU6wlXKvIQjq4caDW'],6vRQJ0VXOl5HiKZNxNM4o9,JC la Amenaza
3,NaN,43I0d2a2aX1CImG6UM1Q3u,https://i.scdn.co/image/ab67616d0000b273c1a1c6...,6km/h,McDog Workas Inc.,8,2024-01-21,day,1,['6km/h'],['6KwBslnDXZS3XTgP4anusf'],3zMbBCuuUd9XkSrBkJjVnz,CHICKEN BLOW THE IDOL
4,NaN,3eMOde5vISAA7biEtF3KXW,https://i.scdn.co/image/ab67616d0000b2734f2df3...,Dunkelblauer Himmel,COFF€€ €NT€RTAINM€NT,0,2022-04-15,day,1,['Dunkelblauer Himmel'],['5vREtDnLI8mphvAJYhVT8D'],3TMA4KovUuQAEaaryhcyHc,LIL KODEX GIGAS


In [45]:
data

In [78]:
(access_token,tiempo_token) = solicitar_token()
# 4.- Tomamos los ids_artist y los ids_tracks del dataframe_albums
ids_artist = set(df_albums['id_artist'] )
ids_tracks=set()
for lista in df_albums['id_tracks']:
    ids_tracks.update(lista)

# 5.- Generar data_artists. Tomamos los ids_artist, los comparamos con los ids que hay en el DataFrame de artist eliminando los duplicados, agrupamos de 100 en 100 (str_ids_artist_100) y hacemos get_artist en bucle
try:
    df_artists = pd.read_csv('df_artists.csv')
except: 
    data_df = list()
    df_artists = pd.DataFrame(data = data_df, columns = ['followers', 'genres', 'id', 'images', 'name', 'popularity'])
   
ids=set(ids_artist)
ids = ids.difference(set(df_artists['id'] ))

# Generar data_artists
str_ids_artist_100 = str()
offset = 0
ids = list(ids)
data_artists=list()
data=list()

progress_bar = tqdm(total=(len(ids) // 100)+1, desc="Processing")
while offset < len(ids):
    str_ids_artist_100 = ','.join( ids[offset:offset+100])
    offset += 100
    
    #comprobar/renovar token
    if time.time()-tiempo_token > 3500:
        (access_token,tiempo_token) = solicitar_token()
    
    #descargamos los datos de los albums de la API y los almacenamos en data_albums
    data = get_artists(access_token,str_ids_artist_100) 
    data_artists.extend(data.get('artists'))
    progress_bar.update(1)
    sleep(0.2)
progress_bar.close()

# 6.- Generar dataframe_artist con el data_artists
df_artists_temp = dataframe_artist(data_artists) 
df_artists = pd.concat([df_artists, df_artists_temp], axis = 0)
df_artists.drop_duplicates(subset=['id'], inplace=True)   
 
 #guardamos el Dataframe como csv:
df_artists.to_csv('df_artists.csv', index=False) 

Processing:   0%|          | 0/687 [06:10<?, ?it/s]


Error en get_tracks: La respuesta de la API no es un JSON válido. Response=(<Response [429]>, {'cache-control': 'private, max-age=0', 'retry-after': '54521', 'access-control-allow-origin': '*', 'access-control-allow-headers': 'Accept, App-Platform, Authorization, Content-Type, Origin, Retry-After, Spotify-App-Version, X-Cloud-Trace-Context, client-token, content-access-token', 'access-control-allow-methods': 'GET, POST, OPTIONS, PUT, DELETE, PATCH', 'access-control-allow-credentials': 'true', 'access-control-max-age': '604800', 'content-encoding': 'gzip', 'strict-transport-security': 'max-age=31536000', 'x-content-type-options': 'nosniff', 'date': 'Wed, 20 Mar 2024 17:07:46 GMT', 'server': 'envoy', 'Via': 'HTTP/2 edgeproxy, 1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'Transfer-Encoding': 'chunked'}) 


AttributeError: 'NoneType' object has no attribute 'get'

In [62]:
df_artists

,followers,genres,id,images,name,popularity


In [48]:
# Solicitud datos album
def get_albums(access_token,str_ids_albums_20):
    # URL de la solicitud GET
    url = "https://api.spotify.com/v1/albums"

    # Parámetros de la solicitud max 20
    params = {
            "ids": str_ids_albums_20
    }

    # Encabezados de la solicitud
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    # Realizar la solicitud GET
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200 and response.content:
        return (response.json())
       
    else:
        print(f'Error en get_albums: La respuesta de la API no es un JSON válido. Response={(response,response.headers)} ')

    

# 

In [49]:
# Solicitud datos track
def get_tracks(access_token,str_ids_tracks_50):

    # URL de la solicitud GET
    url = 'https://api.spotify.com/v1/tracks'

    # Parámetros de la solicitud max 50
    params = {
            "ids": str_ids_tracks_50
    }

    # Encabezados de la solicitud
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    # Realizar la solicitud GET
    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200 and response.content:
        return (response.json())
       
    else:
        print(f'Error en get_tracks: La respuesta de la API no es un JSON válido. Response={(response,response.headers)} ')

    
    



In [50]:
# Solicitud datos artist
def get_artists(access_token,str_ids_artist_100):

    # URL de la solicitud GET
    url = 'https://api.spotify.com/v1/artists'

    # Parámetros de la solicitud 100 max
    params = {
            "ids": str_ids_artist_100
    }

    # Encabezados de la solicitud
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    # Realizar la solicitud GET
    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200 and response.content:
        return (response.json())
       
    else:
        print(f'Error en get_tracks: La respuesta de la API no es un JSON válido. Response={(response,response.headers)} ')



In [26]:
# Esta función genera los dataframes de albums 

def dataframe_albums(data_albums):
    
    data_df = list()
    for album in data_albums:
        try:
            genres                  =   ', '.join(album.get("genres", []))
            id                      =   album.get("id")
            images                  =   album["images"][0].get("url")
            name                    =   album.get("name")
            label                   =   album.get("label")
            popularity              =   album.get("popularity")
            release_date            =   album.get("release_date")
            release_date_precision  =   album.get("release_date_precision")
            total_tracks            =   album.get("total_tracks")
            name_tracks             =   list()
            id_tracks               =   list()
            for track in album['tracks']["items"]:
                name_tracks.append(track.get('name'))
                id_tracks.append(track.get('id'))
            id_artist               =   album["artists"][0].get('id')
            name_artist             =   album["artists"][0].get('name')
            
            data_df.append([genres, id, images, name, label,popularity,release_date ,release_date_precision ,total_tracks ,name_tracks,id_tracks , id_artist,name_artist ])
        
        except:
            pass
    
    df_albums = pd.DataFrame(data = data_df, columns = ['genres', 'id', 'images', 'name', 'label','popularity','release_date' ,'release_date_precision' ,'total_tracks' ,'name_tracks','id_tracks' , 'id_artist','name_artist'])
    
    return df_albums

In [27]:
# Esta función genera los dataframes de tracks 

def dataframe_tracks(data_tracks):
    
    data_df = list()
    for track in data_tracks:
        try:
            disc_number             =   track.get("disc_number")
            duration_ms             =   track.get("duration_ms")
            id                      =   track.get("id")
            name                    =   track.get("name")
            popularity              =   track.get("popularity")
            track_number            =   track.get("track_number")
            id_album                =   track["album"].get('id')
            name_album              =   track["album"].get('name')
            id_artist               =   track["artists"][0].get('id')
            name_artist             =   track["artists"][0].get('name')
            
            data_df.append([disc_number,duration_ms, id, name,popularity,track_number,id_album, name_album, id_artist,name_artist ])
        
        except:
            pass

    df_tracks = pd.DataFrame(data = data_df, columns = ['disc_number','duration_ms', 'id', 'name','popularity','track_number','id_album', 'name_album', 'id_artist','name_artist' ])
    return df_tracks

In [28]:
# Esta función genera los dataframes de artistas

def dataframe_artist(data_artists):
    
    data_df = list()
    for artist in data_artists:
        try:
            followers   =   artist["followers"].get("total")
            genres      =   ', '.join(artist.get("genres", []))
            id          =   artist.get("id")
            images      =   artist["images"][0].get("url")
            name        =   artist.get("name")
            popularity  =   artist.get("popularity")
                    
            data_df.append([followers, genres, id, images, name, popularity])
    
        except:
            pass    
    
    df_artists = pd.DataFrame(data = data_df, columns = ['followers', 'genres', 'id', 'images', 'name', 'popularity'])
    return df_artists
 